In [12]:
import pandas as pd

In [13]:
invoice = pd.read_csv("Invoice (2) - Invoice (2).csv")

In [14]:
invoice = invoice[~invoice["Invoice ID"].duplicated(keep=False)]
invoice = invoice[["Invoice Number", "Customer Name"]]


In [15]:
mis_master = pd.read_excel(
    "MIS_Coresphere100 Apr 25 to Oct 25.xlsx", 
    sheet_name="FY 25-26-Accrual",
    skiprows=2
)

mis_master = mis_master.drop(mis_master.columns[9:], axis=1)

# Standardize column names and align invoice column name to spec
mis_master.columns = mis_master.columns.str.strip()
mis_master = mis_master.rename(columns={"Invoice": "Invoice Number"})

In [16]:
mis_master = pd.concat([mis_master, invoice], ignore_index=True)

In [6]:
data_main = pd.read_excel(
    "MIS_Coresphere100 Apr 25 to Oct 25.xlsx", 
    sheet_name="Invoices",  
    skiprows=2
)

In [7]:
fy_master = pd.read_excel(
    "Copy of Master List of Customers_FY 25-26.xlsx",
    skiprows=2,
    sheet_name="FY 25-26"
)

# Standardize column names (strip spaces)
fy_master.columns = fy_master.columns.str.strip()


In [8]:
def normalize_fy_invoices(fy_master: pd.DataFrame, invoice_col: str = "invoice") -> pd.DataFrame:
    """
    For each row in fy_master[invoice_col]:
      - Split by commas, drop blanks
      - Keep ONLY the last non-empty value
      - Normalize to full invoice number using prefix from the first value
      - Return a DataFrame with a unified column name: "invoice"
    """

    rows = []

    col_invoice = invoice_col if invoice_col in fy_master.columns else invoice_col.capitalize()
    if col_invoice not in fy_master.columns:
        return pd.DataFrame(columns=["invoice"])

    for _, row in fy_master.iterrows():
        raw = row.get(col_invoice)

        if pd.isna(raw):
            continue

        parts = [p.strip() for p in str(raw).split(",") if p.strip() != ""]
        if not parts:
            continue

        first = parts[0]
        if "-" in first:
            prefix, _, suffix = first.rpartition("-")
            prefix = prefix + "-"
            suffix_len = len(suffix)
        else:
            prefix = ""
            suffix_len = len(first)

        last_part = parts[-1]
        if "-" in last_part:
            inv_full = last_part
        else:
            inv_full = prefix + last_part.zfill(suffix_len)

        new_row = row.copy()
        new_row[col_invoice] = inv_full
        rows.append(new_row)

    result = pd.DataFrame(rows)
    if not result.empty and col_invoice != "invoice":
        result = result.rename(columns={col_invoice: "invoice"})
    return result


In [9]:
def enrich_mis_with_fy(mis_master: pd.DataFrame, fy_master: pd.DataFrame) -> pd.DataFrame:
    # Normalize FY invoices (keep only the last invoice per cell and expand short codes)
    fy_norm = normalize_fy_invoices(fy_master, invoice_col="invoice")

    if fy_norm.empty:
        return mis_master

    # Build lookup with needed columns
    base_cols = ["Start Date", "End Date", "Payment Cycle", "Contract Amount"]
    fy_lookup_cols = ["invoice"] + [c for c in base_cols if c in fy_norm.columns]
    fy_lookup = fy_norm[fy_lookup_cols].drop_duplicates(subset=["invoice"])

    # Ensure MIS has Invoice Number column and no trailing spaces
    mis_master = mis_master.copy()
    mis_master.columns = mis_master.columns.str.strip()
    if "Invoice Number" not in mis_master.columns and "Invoice" in mis_master.columns:
        mis_master = mis_master.rename(columns={"Invoice": "Invoice Number"})

    # Merge
    mis_enriched = mis_master.merge(
        fy_lookup,
        left_on="Invoice Number",
        right_on="invoice",
        how="left",
        suffixes=("_mis", "_fy"),
    )

    # Collapse suffixes back to single columns
    for col in base_cols:
        col_mis = f"{col}_mis"
        col_fy = f"{col}_fy"
        if col_mis in mis_enriched and col_fy in mis_enriched:
            mis_enriched[col] = mis_enriched[col_mis].where(
                mis_enriched[col_mis].notna(),
                mis_enriched[col_fy],
            )
            mis_enriched = mis_enriched.drop(columns=[col_mis, col_fy])
        elif col_fy in mis_enriched:
            mis_enriched[col] = mis_enriched[col_fy]
            mis_enriched = mis_enriched.drop(columns=[col_fy])
        elif col_mis in mis_enriched:
            mis_enriched[col] = mis_enriched[col_mis]
            mis_enriched = mis_enriched.drop(columns=[col_mis])

    if "invoice" in mis_enriched.columns:
        mis_enriched = mis_enriched.drop(columns=["invoice"])

    return mis_enriched


In [10]:
mis_master = enrich_mis_with_fy(mis_master, fy_master)

In [11]:
mis_master

,Contract,Months,Customer Name,Invoice Number,Nature of service,Start Date,End Date,Payment Cycle,Contract Amount
0,0,2025-04-01 00:00:00,Touchstone Partners,INV-25-26-000001,subscription based service,2025-04-01 00:00:00,2025-06-30 00:00:00,"3 months subscription, upfront",70800.0
1,Closed- May 25,2025-04-01 00:00:00,Bimal Rajasekhar,INV-25-26-000002,subscription based service,2025-04-01 00:00:00,2025-05-31 00:00:00,2 months- Upfront,35400.0
2,0,2025-04-01 00:00:00,Veritas Legal Advocates and Solicitors,INV-25-26-000003,subscription based service,2025-04-02 00:00:00,2025-10-02 00:00:00,6 month-Upfront,1172448.0
3,0,Apr & May 25,LEGALOGIC CONSULTING LLP,"INV-25-26-000004,104",subscription based service,2025-04-01 00:00:00,2025-05-31 00:00:00,"2 months subscription, Monthly",70800.0
4,0,2025-04-01 00:00:00,VACUITAS COUNSELLOR LLP,INV-25-26-000005,subscription based service,2025-04-03 00:00:00,2026-04-02 00:00:00,Annual Upfront,116820.0
...,...,...,...,...,...,...,...,...,...
558,NaN,NaN,Solicitors India Law Offices,INV-25-26-000573,NaN,2025-10-30 00:00:00,2025-11-29 00:00:00,1 month-Upfront,9440.0
559,NaN,NaN,"Jani & Parikh, Advocates and Solicitors",INV-25-26-000574,NaN,2025-10-30 00:00:00,2025-11-29 00:00:00,1 month-Upfront,21240.0
560,NaN,NaN,Siddhant Makkar,INV-25-26-000575,NaN,2025-10-31 00:00:00,2025-11-30 00:00:00,1 month-Upfront,5000.0
561,NaN,NaN,Astik Vaid,INV-25-26-000576,NaN,2025-10-31 00:00:00,2025-11-30 00:00:00,1 month-Upfront,8000.0
